In [1]:
!pip install xgboost==1.6.1
!pip install scikit-optimize


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

choose between xgbr, dtr and ann (1.rfr ,2.xgbr, 3.ann)

In [3]:
from xgboost import XGBRegressor
new_model=XGBRegressor()
new_model.load_model('/content/drive/MyDrive/Colab Notebooks/xgbr_avggc4in.h5')

#pickle for rfr
#import pickle
#with open('/content/drive/MyDrive/Colab Notebooks/rfr_avggc.pkl' , 'rb') as f:
#    new_model = pickle.load(f)

#from keras import models
# Recreate the exact same model, including its weights and the optimizer(r2/3)
#new_model = models.load_model('/content/drive/MyDrive/Colab Notebooks/my_tikamaveragegcr5-5r6.h5')
# Show the model architecture
#new_model.summary()

In [4]:
import pandas as pd
import numpy as np
test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/average GC full dataset new for colab 1007 without powder mass.csv')
Xtest=test.iloc[:,:4].values
Ytest = test.iloc[:,4:].values
print(Xtest[0])


[2.00000000e+00 3.55247982e-01 3.00000000e+00 1.00000000e+03]


In [5]:
#prediction=new_model.predict(Xtest)
#print(prediction)
#print(Ytest)

#import numpy as np

#Defining MAPE function
#def MAPE(Y_actual,Y_Predicted):
 #   mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
  #  return mape

#LR_MAPE= MAPE(Ytest,prediction)
#print("MAPE: ",LR_MAPE)
#print(Ytest[0,2])

In [6]:
#df=pd.DataFrame(prediction,index=None)
#arr=df[2]
#arr2=arr.to_numpy()
#print(arr2)

In [7]:
target=[(15,10,45)]
weight=[(0.5,0.1,1)]

def prediction(input):
   
   pred=new_model.predict(input)
   preddf=pd.DataFrame(pred,index=None)
   predvalue=preddf[2]
   predarr=predvalue.to_numpy()


   return predarr


In [8]:
def fitness(input):
    ans=prediction(input)
    return ans
  


In [9]:
target=[(15,10,45)]
weight=[(0.5,0.1,1)]
import random
#generate solutions
input=[]
for s in range(20):
    NPCtype=random.randint(1,2)
    
    #NPCweight=random.randrange(50000,61000,100)/1000
    loading=random.randrange(25,60,1)/100
    KOHconc=random.choice([1,3,0.75])
    #ratio=random.randrange(7,11,1)
    current=random.randrange(100,1001,100)


    input.append([(NPCtype,loading,KOHconc,current)])

input=np.asarray(input)
mating_pool=input.reshape(input.shape[0],input.shape[2])



In [10]:
test=prediction(mating_pool)
test=np.asarray(test)

print(mating_pool)
print(test)

[[1.0e+00 4.9e-01 1.0e+00 3.0e+02]
 [2.0e+00 3.8e-01 3.0e+00 1.0e+03]
 [1.0e+00 5.3e-01 1.0e+00 4.0e+02]
 [2.0e+00 4.9e-01 7.5e-01 6.0e+02]
 [2.0e+00 2.5e-01 7.5e-01 8.0e+02]
 [2.0e+00 5.9e-01 1.0e+00 1.0e+02]
 [2.0e+00 5.3e-01 3.0e+00 4.0e+02]
 [2.0e+00 3.8e-01 7.5e-01 5.0e+02]
 [1.0e+00 4.4e-01 1.0e+00 5.0e+02]
 [1.0e+00 5.8e-01 7.5e-01 1.0e+03]
 [2.0e+00 3.9e-01 3.0e+00 3.0e+02]
 [1.0e+00 4.1e-01 1.0e+00 2.0e+02]
 [1.0e+00 2.6e-01 3.0e+00 1.0e+03]
 [2.0e+00 2.6e-01 1.0e+00 1.0e+03]
 [1.0e+00 3.8e-01 7.5e-01 9.0e+02]
 [2.0e+00 3.0e-01 1.0e+00 9.0e+02]
 [1.0e+00 2.5e-01 1.0e+00 8.0e+02]
 [1.0e+00 5.3e-01 7.5e-01 8.0e+02]
 [1.0e+00 5.4e-01 7.5e-01 8.0e+02]
 [1.0e+00 5.1e-01 1.0e+00 9.0e+02]]
[29.445862 33.91476  34.81136  34.385845 30.888504 21.35741  41.47521
 36.440388 30.171047 34.662403 41.03048  24.310978 18.967422 24.71017
 39.330826 24.71017  30.245354 36.023132 36.023132 35.03303 ]


In [11]:
#select first four as parents(best 4)
def parent_selection(mixsort):
    fitness_mating_pool=prediction(mating_pool)
    mix_mating_fitness=np.column_stack((mating_pool,fitness_mating_pool))

    mixsort=mix_mating_fitness[mix_mating_fitness[:, 4].argsort()]
    parents_pool=mixsort[:,0:4]
    parents_results=mixsort[:,4]
    return parents_pool

In [12]:
# crossover(for mating)
def uniform_crossover(A,B,C,D,P):
    for i in range(len(P)):
        if P[i]<0.5:
          temp=A[i]
          A[i]=B[i]
          B[i]=C[i]
          C[i]=D[i]
          D[i]=temp
    return A,B,C,D

def crossover_process(mating_pool):
    A=mating_pool[0]
    B=mating_pool[1]
    C=mating_pool[2]
    D=mating_pool[3]
    P=np.random.rand(4)
    crossover_pool=uniform_crossover(A,B,C,D,P)
    crossover_pool=np.asarray(crossover_pool)
    return crossover_pool


#crossover_pool=crossover_process(parents_pool)
#print(crossover_pool)



In [13]:
#mutation!
def mutation(crossover_pool):
    mutation_rate=random.uniform(0.8,1.2)
    mutation_X=crossover_pool[:,0]
    mutation_pool=crossover_pool[:,1:]*mutation_rate
    final_mutation=np.column_stack((mutation_X,mutation_pool))
    return final_mutation

#mutation_pool=mutation(crossover_pool)
#print(mutation_pool)
#print(mutation_pool)
#mutation_rate=random.uniform(0.98,1.02)
#mutation_pool=crossover_pool*mutation_rate


In [14]:
#survival selection by def
def survivor_selection():
    new_mating_pool=np.insert(mutation(crossover_process(parent_selection(mating_pool))),0,mating_pool,axis=0)
    fitness_new_pool=fitness(new_mating_pool)
    mix_new_fitness=np.column_stack((new_mating_pool,fitness_new_pool))

    mixsortnew=mix_new_fitness[mix_new_fitness[:, 4].argsort()]
    mixsortnew =mixsortnew[::-1]
    mixsortnew=mixsortnew[:]
    new_mating_pool=mixsortnew[0:20,0:4]
    return new_mating_pool


test2=survivor_selection()
print(test2)

[[2.00000000e+00 5.30000000e-01 3.00000000e+00 4.00000000e+02]
 [2.00000000e+00 3.90000000e-01 3.00000000e+00 3.00000000e+02]
 [1.00000000e+00 3.80000000e-01 7.50000000e-01 9.00000000e+02]
 [2.00000000e+00 3.05903219e-01 1.17655084e+00 1.17655084e+03]
 [2.00000000e+00 3.80000000e-01 7.50000000e-01 5.00000000e+02]
 [1.00000000e+00 5.40000000e-01 7.50000000e-01 8.00000000e+02]
 [1.00000000e+00 5.30000000e-01 7.50000000e-01 8.00000000e+02]
 [1.00000000e+00 5.10000000e-01 1.00000000e+00 9.00000000e+02]
 [1.00000000e+00 5.30000000e-01 1.00000000e+00 4.00000000e+02]
 [1.00000000e+00 5.80000000e-01 7.50000000e-01 1.00000000e+03]
 [1.00000000e+00 3.52965253e-01 1.17655084e+00 1.05889576e+03]
 [2.00000000e+00 4.90000000e-01 7.50000000e-01 6.00000000e+02]
 [2.00000000e+00 3.80000000e-01 3.00000000e+00 1.00000000e+03]
 [2.00000000e+00 2.50000000e-01 7.50000000e-01 8.00000000e+02]
 [1.00000000e+00 2.50000000e-01 1.00000000e+00 8.00000000e+02]
 [1.00000000e+00 4.40000000e-01 1.00000000e+00 5.000000

In [15]:
def genetic_algorithm():
    parents=parent_selection(mating_pool)
    
    
    cross_p=crossover_process(parents)
   
    mutant_p=mutation(cross_p)
    survived_p=survivor_selection()
    return survived_p
    


print(genetic_algorithm())


[[2.00000000e+00 5.30000000e-01 3.00000000e+00 4.00000000e+02]
 [2.00000000e+00 3.90000000e-01 3.00000000e+00 3.00000000e+02]
 [1.00000000e+00 3.80000000e-01 7.50000000e-01 9.00000000e+02]
 [2.00000000e+00 3.80000000e-01 7.50000000e-01 5.00000000e+02]
 [1.00000000e+00 5.40000000e-01 7.50000000e-01 8.00000000e+02]
 [1.00000000e+00 5.30000000e-01 7.50000000e-01 8.00000000e+02]
 [1.00000000e+00 6.90962131e-01 1.17112226e+00 1.17112226e+03]
 [1.00000000e+00 5.10000000e-01 1.00000000e+00 9.00000000e+02]
 [1.00000000e+00 5.30000000e-01 1.00000000e+00 4.00000000e+02]
 [1.00000000e+00 5.80000000e-01 7.50000000e-01 1.00000000e+03]
 [2.00000000e+00 4.90000000e-01 7.50000000e-01 6.00000000e+02]
 [2.00000000e+00 3.80000000e-01 3.00000000e+00 1.00000000e+03]
 [2.00000000e+00 3.04491786e-01 3.51336677e+00 1.05401003e+03]
 [2.00000000e+00 2.50000000e-01 7.50000000e-01 8.00000000e+02]
 [1.00000000e+00 2.50000000e-01 1.00000000e+00 8.00000000e+02]
 [1.00000000e+00 4.40000000e-01 1.00000000e+00 5.000000

In [16]:
initial_gen=0
max_gen=1000



while initial_gen<max_gen:
   x_new=genetic_algorithm()
   initial_gen+=1 
   mating_pool=x_new
   
   first=x_new[0]
   first=np.asarray(first)
   print([first])
   print("gen",initial_gen,"=",x_new)
   
   test=new_model.predict(x_new)
   c2test=test[:,2]+test[:,4]
   print("best results:",test[0])
   
   print("Fittest C2H4+ethanol:",c2test[0])
   if c2test[0]>=75 : break
   

Streaming output truncated to the last 5000 lines.
 [2.00000000e+00 3.58179309e-01 3.64826762e+00 8.89695222e+02]
 [1.00000000e+00 3.55287838e-01 8.78125768e-01 7.75411813e+02]
 [2.00000000e+00 3.59136635e-01 3.05738836e+00 8.73039969e+02]
 [2.00000000e+00 3.57940610e-01 9.06936040e-01 8.87591880e+02]
 [2.00000000e+00 3.57621155e-01 9.06126618e-01 8.84662468e+02]
 [2.00000000e+00 3.56633995e-01 9.04313975e-01 8.82892763e+02]
 [2.00000000e+00 3.57630102e-01 9.08315728e-01 8.86799722e+02]]
best results: [12.321926  22.53278   47.694347   1.5796994 16.920021 ]
Fittest C2H4+ethanol: 64.614365
[array([2.00000000e+00, 3.54026214e-01, 8.27471755e-01, 8.63323703e+02])]
gen 784 = [[2.00000000e+00 3.54026214e-01 8.27471755e-01 8.63323703e+02]
 [2.00000000e+00 3.53707201e-01 7.25439979e-01 8.84342455e+02]
 [2.00000000e+00 3.54017788e-01 7.21630120e-01 8.79698073e+02]
 [2.00000000e+00 3.55328191e-01 8.74719441e-01 8.82613222e+02]
 [1.00000000e+00 3.57315499e-01 7.13079698e-01 8.09326509e+02]
 [1.0

In [17]:
print(test[0],mating_pool[0])
print(new_model.predict(mating_pool))


[12.321926  22.53278   47.694347   1.5796994 16.920021 ] [2.00000000e+00 3.54026214e-01 8.27471755e-01 8.63323703e+02]
[[12.321926  22.53278   47.694347   1.5796994 16.920021 ]
 [12.321926  22.53278   47.694347   1.5796994 16.920021 ]
 [12.321926  22.53278   47.694347   1.5796994 16.920021 ]
 [12.321926  22.53278   47.694347   1.5796994 16.920021 ]
 [12.128312  22.618183  47.623802   1.6584038 15.18746  ]
 [12.128312  22.618183  47.623802   1.6584038 15.18746  ]
 [13.073109  22.322866  47.599743   1.3643461 16.920021 ]
 [12.866941  22.442686  47.572254   3.736539  16.001585 ]
 [14.767156  14.422037  47.44177    1.6188856 17.93812  ]
 [16.616426  13.820398  47.439896   0.9740801 16.062798 ]
 [21.862898  11.947439  46.673653   1.340844  20.142757 ]
 [21.862898  11.947439  46.673653   1.340844  20.142757 ]
 [13.730603  22.905165  46.638134   1.7443068 15.240268 ]
 [ 9.63704   22.859104  46.606026   1.6271006 15.116372 ]
 [15.399693  21.059488  46.556267   1.6699343 16.596846 ]
 [ 9.63704 